## Import Packages & Run Queries of Base Data

In [16]:
import rhutils as ds
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

secret saved to /opt/app-root/src/.config/gspread_pandas/google_secret.json
secret saved to /opt/app-root/src/.config/gspread_pandas/creds/jgooley


In [2]:
pipe = ds.query_RS("""
(SELECT 
    snapshot_date,
    CASE WHEN product_line LIKE '%TAM%' THEN 'TAM' ELSE product_forecast_group END AS cs_product_group,
    reporting_services_geo,
    account_segment,
    snapshot_date_fiscal_year_quarter_q close_yr_q,
    snapshot_date_day_of_fiscal_quarter_number day_of_q,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Closed' THEN syb_amount_usd_cy_pr ELSE 0 END),0) closed_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Commit' THEN syb_amount_usd_cy_pr ELSE 0 END),0) commit_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Best Case' THEN syb_amount_usd_cy_pr ELSE 0 END),0) bestcase_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Pipeline' THEN syb_amount_usd_cy_pr ELSE 0 END),0) pipeline_syb
FROM bf_ard.core_global_pipeline_snapshot_daily_pq
WHERE 
    cs_product_group IN ('CONSULTING', 'TRAINING','TAM')
    AND opportunity_split_type_h = 'Bookings'
    AND snapshot_date >= '2020-04-01'
    AND reporting_services_geo IN ('NA Comm','NA Public Sector','LATAM', 'EMEA', 'APAC')
    AND snapshot_date_fiscal_year_quarter_q = opportunity_close_date_fiscal_year_quarter_q
GROUP BY 1,2,3,4,5,6)
UNION
(SELECT 
    snapshot_date,
    CASE WHEN product_line LIKE '%TAM%' THEN 'TAM' ELSE product_forecast_group END AS cs_product_group,
    reporting_services_geo,
    account_segment,
    snapshot_date_fiscal_year_quarter_q close_yr_q,
    snapshot_date_day_of_fiscal_quarter_number day_of_q,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Closed' THEN syb_amount_usd_cy_pr ELSE 0 END),0) closed_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Commit' THEN syb_amount_usd_cy_pr ELSE 0 END),0) commit_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Best Case' THEN syb_amount_usd_cy_pr ELSE 0 END),0) bestcase_syb,
    ROUND(SUM(CASE WHEN opportunity_forecast_category_h = 'Pipeline' THEN syb_amount_usd_cy_pr ELSE 0 END),0) pipeline_syb
FROM bf_ard.core_global_pipeline_snapshot_daily_cq
WHERE 
    cs_product_group IN ('CONSULTING', 'TRAINING','TAM')
    AND opportunity_split_type_h = 'Bookings'
    AND reporting_services_geo IN ('NA Comm','NA Public Sector','LATAM', 'EMEA', 'APAC')
    AND snapshot_date_fiscal_year_quarter_q = opportunity_close_date_fiscal_year_quarter_q
GROUP BY 1,2,3,4,5,6)
ORDER BY 1,2,3,4,5,6""")

pipe['snapshot_date'] = pd.to_datetime(pipe['snapshot_date'])
pipe['open_pipe'] = pipe['commit_syb'] + pipe['bestcase_syb'] + pipe['pipeline_syb']

Using default environment variable DS_RS_USER
Using default environment variable DS_RS_PASS


In [3]:
actuals = ds.query_RS("""
SELECT
    CASE WHEN product_line LIKE '%TAM%' THEN 'TAM' ELSE product_forecast_group END AS cs_product_group,
    reporting_services_geo,
    account_segment_h,
    opportunity_close_date_fiscal_year_quarter_q close_yr_q,
    ROUND(SUM(syb_amount_usd_cy_pr),0) actual_syb
FROM bf_ard.core_global_historical_bookings
WHERE 
    cs_product_group IN ('CONSULTING', 'TRAINING','TAM')
    AND opportunity_close_date >= '2020-04-01'
    AND reporting_services_geo IN ('NA Comm','NA Public Sector','LATAM', 'EMEA', 'APAC')
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4""")

Using default environment variable DS_RS_USER
Using default environment variable DS_RS_PASS


In [4]:
targets = ds.query_RS("""
SELECT
    forecast_product_group,
    CASE 
        WHEN geo = 'North America' AND region = 'NAPS' THEN 'NA Public Sector' 
        WHEN geo = 'North America' AND region <> 'NAPS' THEN 'NA Comm'
        ELSE geo
    END AS geo,
    segment,
    calendar_year || '-Q' || calendar_qtr date_qtr,
    ROUND(SUM(syb_usd__rh_plan_rate_)) target_syb
FROM rsds_ops_planning.target_bookings_calendar_year
WHERE forecast_product_group IN ('CONSULTING','TRAINING')
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4""")

#TAM
tam_targets = pd.DataFrame([['TAM','NA Comm','2022-Q1',6400703],
['TAM','NA Comm','2022-Q2',3901582],
['TAM','NA Comm','2022-Q3',4745167],
['TAM','NA Comm','2022-Q4',6917399],
['TAM','NA Public Sector','2022-Q1',3343839],
['TAM','NA Public Sector','2022-Q2',1010653],
['TAM','NA Public Sector','2022-Q3',3265226],
['TAM','NA Public Sector','2022-Q4',2475684],
['TAM','EMEA','2022-Q1',3357002],
['TAM','EMEA','2022-Q2',3538016],
['TAM','EMEA','2022-Q3',3916502],
['TAM','EMEA','2022-Q4',6960842],
['TAM','APAC','2022-Q1',2837915],
['TAM','APAC','2022-Q2',3165497],
['TAM','APAC','2022-Q3',3129488],
['TAM','APAC','2022-Q4',3593451],
['TAM','LATAM','2022-Q1',2703608],
['TAM','LATAM','2022-Q2',1478536],
['TAM','LATAM','2022-Q3',1520780],
['TAM','LATAM','2022-Q4',1816487]],columns = ['cs_product_group','geo','date_qtr','target_syb']
)

Using default environment variable DS_RS_USER
Using default environment variable DS_RS_PASS


In [5]:
quarter_start = pd.to_datetime(pd.datetime.today() - pd.tseries.offsets.QuarterBegin(startingMonth=1))
quarter_start = quarter_start.normalize()

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



## Run All Services Forecasts

In [6]:
#Filter Data to Services Only, Merge Data
services_pipe = pipe.copy()[pipe['cs_product_group'].isin(['CONSULTING','TRAINING'])]
services_actuals = actuals.copy()[actuals['cs_product_group'].isin(['CONSULTING','TRAINING'])]
services_targets = targets.copy()[targets['forecast_product_group'].isin(['CONSULTING','TRAINING'])]
services_df = services_pipe.merge(services_actuals,how='left',
                                  left_on=['cs_product_group','close_yr_q','reporting_services_geo','account_segment'],
                                 right_on=['cs_product_group','close_yr_q','reporting_services_geo','account_segment_h'])
services_df = services_df.merge(services_targets,how='left',
                                left_on=['cs_product_group','close_yr_q','reporting_services_geo','account_segment'],
                                 right_on=['forecast_product_group','date_qtr','geo','segment'])
services_df['final_syb'] = np.where(services_df['snapshot_date']>=quarter_start,services_df['target_syb'],services_df['actual_syb'])
services_df['quarter_number'] = services_df['close_yr_q'].str[-1:]
services_df = services_df[['snapshot_date','cs_product_group','reporting_services_geo','account_segment',
                           'close_yr_q','quarter_number','day_of_q','closed_syb','commit_syb','bestcase_syb',
                           'pipeline_syb','open_pipe','final_syb']][~services_df['account_segment'].isin(['Unsegmented',None])]

In [7]:
#Define Function to get Power Set
def powerset(s):
    x = len(s)
    for i in range(1 << x):
        yield [s[j] for j in range(x) if (i & (1 << j))]

In [8]:
#Create Services Dataframe with all LODs
df_list = []

for x in list(powerset(['cs_product_group','reporting_services_geo','account_segment'])):
    group = ['snapshot_date','close_yr_q','quarter_number','day_of_q'] + x
    data = services_df.copy().groupby(by=group).sum().reset_index()
    s = ''
    if 'reporting_services_geo' in x:
        s += 'Geo, '
    else:
        s += 'Global, '
        data['reporting_services_geo'] = 'Global'
        
    if 'cs_product_group' in x:
        s += 'GLS/GPS, '
    else:
        s += 'Services, '
        data['cs_product_group'] = 'SERVICES'
        
    if 'account_segment' in x:
        s += 'Segment'
    else:
        s += 'All Segments'
        data['account_segment'] = 'All Segments'
        
    data['level_of_detail'] = s
    df_list.append(data)
    
df_final_services = pd.concat(df_list)

In [9]:
#Calculate Multipliers & Forecasts
df_final_services['multiplier'] = np.where((df_final_services['final_syb']-df_final_services['closed_syb'])<0,0,((df_final_services['final_syb']-df_final_services['closed_syb'])/df_final_services['open_pipe']))
df_final_services['multiplier'] = df_final_services['multiplier'].fillna(0)

df_final_services = df_final_services.sort_values(['cs_product_group','reporting_services_geo','account_segment','quarter_number','day_of_q','close_yr_q'])
df_final_services['multiplier_ma_2y'] = df_final_services.groupby(by=['cs_product_group','reporting_services_geo','account_segment','quarter_number','day_of_q'])['multiplier'].transform(lambda x: x.rolling(2,1).mean())
df_final_services['forecast_2y'] = df_final_services['closed_syb'] + df_final_services['multiplier_ma_2y'] * df_final_services['open_pipe']
df_final_services['ape_2y'] = abs(df_final_services['forecast_2y']-df_final_services['final_syb'])/df_final_services['final_syb']

df_final_services = df_final_services.sort_values(['cs_product_group','reporting_services_geo','account_segment','day_of_q','close_yr_q'])
df_final_services['multiplier_ma_4q'] = df_final_services.groupby(by=['cs_product_group','reporting_services_geo','account_segment','day_of_q'])['multiplier'].transform(lambda x: x.rolling(4,1).mean())
df_final_services['forecast_4q'] = df_final_services['closed_syb'] + df_final_services['multiplier_ma_4q'] * df_final_services['open_pipe']
df_final_services['ape_4q'] = abs(df_final_services['forecast_4q']-df_final_services['final_syb'])/df_final_services['final_syb']

## Run All TAM Forecasts

In [10]:
#Filter Data to TAM Only, Merge Data
tam_pipe = pipe.copy()[pipe['cs_product_group'].isin(['TAM'])].groupby(by=['snapshot_date','cs_product_group','reporting_services_geo','close_yr_q','day_of_q']).sum().reset_index()
tam_actuals = actuals.copy()[actuals['cs_product_group'].isin(['TAM'])].groupby(by=['cs_product_group','reporting_services_geo','close_yr_q']).sum().reset_index()
tam_targets = tam_targets.copy()
tam_actuals
tam_df = tam_pipe.merge(tam_actuals,how='left',
                                  left_on=['cs_product_group','close_yr_q','reporting_services_geo'],
                                 right_on=['cs_product_group','close_yr_q','reporting_services_geo'])
tam_df = tam_df.merge(tam_targets,how='left',
                                left_on=['cs_product_group','close_yr_q','reporting_services_geo'],
                                 right_on=['cs_product_group','date_qtr','geo'])
tam_df['final_syb'] = np.where(tam_df['snapshot_date']>=quarter_start,tam_df['target_syb'],tam_df['actual_syb'])
tam_df['quarter_number'] = tam_df['close_yr_q'].str[-1:]
tam_df = tam_df[['snapshot_date','cs_product_group','reporting_services_geo',
                           'close_yr_q','quarter_number','day_of_q','closed_syb','commit_syb','bestcase_syb',
                           'pipeline_syb','open_pipe','final_syb']]

In [11]:
#Create TAM Dataframe with all LODs
df_list = []

for x in [[], ['reporting_services_geo']]:
    group = ['snapshot_date','close_yr_q','quarter_number','day_of_q','cs_product_group'] + x
    data = tam_df.copy().groupby(by=group).sum().reset_index()
    s = ''
    if 'reporting_services_geo' in x:
        s += 'Geo, '
    else:
        s += 'Global, '
        data['reporting_services_geo'] = 'Global'
        
    if 'cs_product_group' in x:
        s += 'GLS/GPS, '
    else:
        s += 'TAM, '
        data['cs_product_group'] = 'TAM'
        
    if 'account_segment' in x:
        s += 'Segment'
    else:
        s += 'All Segments'
        data['account_segment'] = 'All Segments'
        
    data['level_of_detail'] = s
    df_list.append(data)
    
df_final_tam = pd.concat(df_list)

In [12]:
#Calculate Multipliers & Forecasts
df_final_tam['multiplier'] = np.where((df_final_tam['final_syb']-df_final_tam['closed_syb'])<0,0,((df_final_tam['final_syb']-df_final_tam['closed_syb'])/df_final_tam['open_pipe']))
df_final_tam['multiplier'] = df_final_tam['multiplier'].fillna(0)

df_final_tam = df_final_tam.sort_values(['cs_product_group','reporting_services_geo','account_segment','quarter_number','day_of_q','close_yr_q'])
df_final_tam['multiplier_ma_2y'] = df_final_tam.groupby(by=['cs_product_group','reporting_services_geo','account_segment','quarter_number','day_of_q'])['multiplier'].transform(lambda x: x.rolling(2,1).mean())
df_final_tam['forecast_2y'] = df_final_tam['closed_syb'] + df_final_tam['multiplier_ma_2y'] * df_final_tam['open_pipe']
df_final_tam['ape_2y'] = abs(df_final_tam['forecast_2y']-df_final_tam['final_syb'])/df_final_tam['final_syb']

df_final_tam = df_final_tam.sort_values(['cs_product_group','reporting_services_geo','account_segment','day_of_q','close_yr_q'])
df_final_tam['multiplier_ma_4q'] = df_final_tam.groupby(by=['cs_product_group','reporting_services_geo','account_segment','day_of_q'])['multiplier'].transform(lambda x: x.rolling(4,1).mean())
df_final_tam['forecast_4q'] = df_final_tam['closed_syb'] + df_final_tam['multiplier_ma_4q'] * df_final_tam['open_pipe']
df_final_tam['ape_4q'] = abs(df_final_tam['forecast_4q']-df_final_tam['final_syb'])/df_final_tam['final_syb']

In [13]:
df_final = pd.concat([df_final_services,df_final_tam])
df_final

,snapshot_date,close_yr_q,quarter_number,day_of_q,closed_syb,commit_syb,bestcase_syb,pipeline_syb,open_pipe,final_syb,...,cs_product_group,account_segment,level_of_detail,multiplier,multiplier_ma_2y,forecast_2y,ape_2y,multiplier_ma_4q,forecast_4q,ape_4q
0,2020-04-01,2020-Q2,2,1,0.0,2275254.0,10833262.0,27145573.0,40254089.0,13204194.0,...,CONSULTING,All Segments,"Geo, GLS/GPS, All Segments",0.328021,0.328021,1.320419e+07,1.410647e-16,0.328021,1.320419e+07,1.410647e-16
910,2020-07-01,2020-Q3,3,1,57065.0,3500904.0,8382577.0,34346412.0,46229893.0,20542702.0,...,CONSULTING,All Segments,"Geo, GLS/GPS, All Segments",0.443125,0.443125,2.054270e+07,0.000000e+00,0.385573,1.788208e+07,1.295169e-01
1830,2020-10-01,2020-Q4,4,1,79120.0,3915957.0,8006227.0,36021770.0,47943954.0,18019844.0,...,CONSULTING,All Segments,"Geo, GLS/GPS, All Segments",0.374202,0.374202,1.801984e+07,0.000000e+00,0.381783,1.838330e+07,2.016972e-02
2750,2021-01-01,2021-Q1,1,1,0.0,2399814.0,6134271.0,35661315.0,44195400.0,18315250.0,...,CONSULTING,All Segments,"Geo, GLS/GPS, All Segments",0.414415,0.414415,1.831525e+07,0.000000e+00,0.389941,1.723360e+07,5.905752e-02
3650,2021-04-01,2021-Q2,2,1,0.0,1816555.0,13305099.0,39669382.0,54791036.0,23058049.0,...,CONSULTING,All Segments,"Geo, GLS/GPS, All Segments",0.420836,0.374429,2.051533e+07,1.102745e-01,0.413145,2.263663e+07,1.827660e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,2020-09-30,2020-Q3,3,92,2311960.0,0.0,0.0,0.0,0.0,2311960.0,...,TAM,All Segments,"Geo, TAM, All Segments",0.000000,0.000000,2.311960e+06,0.000000e+00,0.000000,2.311960e+06,0.000000e+00
1374,2020-12-31,2020-Q4,4,92,1978451.0,0.0,0.0,0.0,0.0,1978451.0,...,TAM,All Segments,"Geo, TAM, All Segments",0.000000,0.000000,1.978451e+06,0.000000e+00,0.000000,1.978451e+06,0.000000e+00
2739,2021-09-30,2021-Q3,3,92,2848980.0,0.0,0.0,0.0,0.0,2848980.0,...,TAM,All Segments,"Geo, TAM, All Segments",0.000000,0.000000,2.848980e+06,0.000000e+00,0.000000,2.848980e+06,0.000000e+00
3199,2021-12-31,2021-Q4,4,92,1972762.0,0.0,114782.0,0.0,114782.0,1972762.0,...,TAM,All Segments,"Geo, TAM, All Segments",0.000000,0.000000,1.972762e+06,0.000000e+00,0.000000,1.972762e+06,0.000000e+00


In [14]:
#Write to redshift
df_final.to_redshift('pipeline_multiplier_forecast',if_exists="replace",truncate=True,schema='services_finance')

Using default environment variable DS_RS_USER
Using default environment variable DS_RS_PASS
Using ~/.aws/credentials to obtain aws access keys
Using ~/.aws/s3_folder to obtain s3 folder
bucket where dataframe is to be copied: rh-stage-ssa
keypath of file to be copied:gfa_svc/pipeline_multiplier_forecast.gzip
Writing to disk...
Updating table services_finance.pipeline_multiplier_forecast
Full S3 url: rh-stage-ssa/gfa_svc/pipeline_multiplier_forecast.gzip
Using default environment variable DS_RS_USER
Using default environment variable DS_RS_PASS
Querying Redshift...
drop table services_finance.pipeline_multiplier_forecast;

CREATE TABLE services_finance.pipeline_multiplier_forecast (
	snapshot_date TIMESTAMP WITHOUT TIME ZONE, 
	close_yr_q TEXT, 
	quarter_number TEXT, 
	day_of_q BIGINT, 
	closed_syb FLOAT(53), 
	commit_syb FLOAT(53), 
	bestcase_syb FLOAT(53), 
	pipeline_syb FLOAT(53), 
	open_pipe FLOAT(53), 
	final_syb FLOAT(53), 
	reporting_services_geo TEXT, 
	cs_product_group TEXT, 
	

In [15]:
max(df_final.snapshot_date)

Timestamp('2022-10-03 00:00:00')